In [ ]:
import asyncio
from agent_framework import ChatMessage, TextContent, Role
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ Importaciones completadas")

## Creación del Agente Conversacional

In [ ]:
agent = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("MODEL")
).create_agent(
    name="MultiturnAgent",
    instructions="Eres un agente útil que cuenta buenos chistes y mantiene conversaciones amenas.",
)

# Crear un thread para mantener el historial
thread = agent.get_new_thread()

print("✅ Agente creado")
print("🧵 Thread inicializado para rastrear conversación")

## Función de Conversación Interactiva

### Características:
- **Loop continuo**: Acepta múltiples entradas
- **Comando de salida**: Escribir "salir" para terminar
- **Thread persistente**: Mantiene contexto entre mensajes

In [ ]:
async def interactive_conversation():
    """
    Loop interactivo de conversación con el agente.
    
    NOTA: Esta versión es para entornos interactivos.
    En un notebook, usa la versión de demostración abajo.
    """
    print("💬 Iniciando conversación interactiva")
    print("📝 Escribe 'salir' para terminar\n")
    print("="*60)
    
    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "salir":
            print("👋 Terminando la conversación.")
            break

        response = await agent.run(
            ChatMessage(
                role=Role.USER,
                contents=[TextContent(text=user_input)]
            ),
            thread=thread
        )
        print(f"Agente: {response.text}\n")

print("✅ Función interactiva definida")

## Demo: Conversación Simulada

Simulemos una conversación de múltiples turnos para demostrar el contexto persistente:

In [ ]:
async def demo_conversation():
    """Demuestra una conversación con contexto persistente."""
    
    print("🎭 Demostración de conversación multi-turno\n")
    print("="*60)
    
    # Turno 1
    print("👤 Usuario: Cuéntame un chiste sobre programadores")
    r1 = await agent.run(
        ChatMessage(role=Role.USER, contents=[TextContent(text="Cuéntame un chiste sobre programadores")]),
        thread=thread
    )
    print(f"🤖 Agente: {r1.text}\n")
    
    # Turno 2 - Referencias el chiste anterior
    print("👤 Usuario: ¿Tienes otro similar?")
    r2 = await agent.run(
        ChatMessage(role=Role.USER, contents=[TextContent(text="¿Tienes otro similar?")]),
        thread=thread
    )
    print(f"🤖 Agente: {r2.text}\n")
    
    # Turno 3 - Cambio de tema
    print("👤 Usuario: Ahora cuéntame uno sobre Python")
    r3 = await agent.run(
        ChatMessage(role=Role.USER, contents=[TextContent(text="Ahora cuéntame uno sobre Python")]),
        thread=thread
    )
    print(f"🤖 Agente: {r3.text}\n")
    
    # Turno 4 - Referencia múltiples contextos previos
    print("👤 Usuario: ¿Cuál de todos los chistes te gustó más?")
    r4 = await agent.run(
        ChatMessage(role=Role.USER, contents=[TextContent(text="¿Cuál de todos los chistes te gustó más?")]),
        thread=thread
    )
    print(f"🤖 Agente: {r4.text}\n")
    
    print("="*60)
    print("\n✅ Demo completada")
    print(f"📊 Total de mensajes en thread: {len(thread.messages)}")

await demo_conversation()

## Inspección del Thread

Veamos qué contiene el thread después de la conversación:

In [ ]:
print("🔍 Contenido del Thread:\n")
print(f"Total de mensajes: {len(thread.messages)}")
print("\nÚltimos 3 mensajes:")
print("="*60)

for i, msg in enumerate(thread.messages[-3:], 1):
    role = "Usuario" if msg.role == Role.USER else "Agente"
    text = msg.contents[0].text if msg.contents else "[sin contenido]"
    print(f"\n{i}. {role}:")
    print(f"   {text[:100]}..." if len(text) > 100 else f"   {text}")

print("\n" + "="*60)

## Análisis y Conclusiones

### Ventajas de Multi-Turn Conversations:

1. **Contexto Natural**: El agente recuerda toda la conversación
2. **Referencias Anafóricas**: Puede entender "ese", "el anterior", etc.
3. **Continuidad**: Las conversaciones fluyen naturalmente
4. **Personalización**: Puede recordar preferencias mencionadas

### Casos de Uso:

1. **Chatbots de Soporte**: Resolución de problemas complejos
2. **Asistentes Personales**: Tareas que requieren múltiples pasos
3. **Tutores Educativos**: Enseñanza paso a paso
4. **Análisis Interactivo**: Exploración de datos con preguntas de seguimiento

### Consideraciones:

1. **Límites de Tokens**: Conversaciones muy largas pueden exceder el límite
2. **Costo**: Cada mensaje incluye todo el historial previo
3. **Memoria**: El thread crece con cada intercambio

### Estrategias de Optimización:

```python
# Limitar historia a últimos N mensajes
MAX_HISTORY = 10
if len(thread.messages) > MAX_HISTORY:
    thread.messages = thread.messages[-MAX_HISTORY:]

# Resumir historial antiguo
if len(thread.messages) > 20:
    summary = await summarize_history(thread.messages[:-10])
    thread.messages = [summary] + thread.messages[-10:]
```